In [7]:
import csp
from csp import Constraint, CSP

In [8]:
from typing import Dict, List, Optional

### Tasks for Artificial Intelligence

Ciara is looking for employees for her new company, which develops and provides AI based logistic software for retailers. Ciara has determined that she needs:

2 Python Programmers, 2 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.
Assume that if a person has two abilities, he or she can take on two roles in the company.

So Ciara narrowed down her selections to the following people:

| Name  | Abilities |
|:-----:|:---------:|
| Peter | Python and AI |
| Juan  | Web and AI |
| Jim   | AI and Systems |
| Jane  | Python and Database |
| Mary  | Web and Systems |
| Bruce | Systems and Python |
| Anita | Web and AI |


Scenario 1:

Suppose Ciara knows Python, and only has funds to hire three more people.


## Scenario 1


In this particular problem, there are many ways to distribute roles. In Constraint Satisfaction Problem (CSP), there are three main elements: Variables(X), Domain(D) and Constraints(Rules). One of the most obvious ways would be to define the elements in the context of a CSP in the following way:

- **Variables (X)**: These are the entities whose values need to be determined. In this case, the variables would be the roles Ciara needs to fill in her company, i.e., Python Programmer, AI Engineer, Web Designer, Database Admin, and Systems Engineer.

- **Domain**: This is the set of possible values that the variables can take. Here, the domain would be the set of people Ciara can hire for each role, i.e., for Python Developer position we would have domains like ('Peter', 'Jane', 'Bruce', 'Ciara')since Ciara knows python we need to include her as well .

- **Constraints (Factors)**: They are basically the rules/restrictions that need to be satisfied in order to assign the domain value to a variable. In this case constraints are:
    1. Each role must be filled by someone who has the corresponding ability.
    (I won't include this particular constraint, since we can avoid it, by simply not adding people to a variable's domain, if those people won't have a corresponding ability) 
    2. One person cannot work in the same position
    3. Ciara can only hire three more people.

Since none of the applicants has 3 or more skills, the constraint “that a person cannot hold more than 2 positions” not need to be written.

This forms the basis of the CSP: 

    'Python Programmer 1': ['Peter', 'Jane', 'Bruce', 'Ciara']
    'Python Programmer 2': ['Peter', 'Jane', 'Bruce', 'Ciara']
    'AI Engineer 1': ['Jim', 'Juan', 'Peter', 'Anita']
    'AI Engineer 2': ['Jim', 'Juan', 'Peter', 'Anita']
    'Web Designer': ['Juan', 'Mary', 'Anita']
    'Database Admin': ['Jane']
    'Systems Engineer': ['Jim', 'Mary', 'Bruce']

## Solution 1


In [12]:
# One person cannot work in the same position 
class HiringLimitConstraint(Constraint[str, str]):
    def __init__(self, role1: str, role2: str):
        super().__init__([role1, role2])
        self.role1: str = role1
        self.role2: str = role2
            
    def satisfied(self, assignment: Dict[str, str]) -> bool:
        """If either self.role1 or self.role2 has not been assigned yet 
        (i.e., they are not keys in the assignment dictionary), 
        then it’s impossible for the same person to have been assigned to both roles, 
        because at least one of the roles hasn’t been assigned yet.
        Therefore, the constraint is satisfied, and the function returns True."""
        
        # Check if the roles have been assigned yet
        role1_assigned = self.role1 in assignment
        role2_assigned = self.role2 in assignment

        # If either role has not been assigned yet, there can't be a conflict
        if not role1_assigned or not role2_assigned:
            return True
        # If both roles have been assigned we check if they have been assigned with the same person
        
        # If the same person were hired to a certain two positions, return False
        return assignment[self.role1] != assignment[self.role2]
    

In [13]:
# Ciara can only hire three more people.
class HiringLimitConstraint2(Constraint[str, str]):
    def __init__(self, variables: List[str]) -> None:
        super().__init__(variables)
        self.variables: List[str] = variables
            
    def satisfied(self, assignment: Dict[str, str]) -> bool:
        # if len(assignment) == len(self.variables):
            # Count the total number of unique people hired
            unique_people = set(assignment.values())
            # Check if more than 4 people are hired (4 including Ciara)
            #print(unique_people)
            return len(unique_people) <= 4


In [14]:
if __name__ == "__main__": 
    # Define the variables (roles)
    variables = ['Python Programmer 1',
                 'Python Programmer 2', 
                 'AI Engineer 1', 
                 'AI Engineer 2', 
                 'Web Designer', 
                 'Database Admin', 
                 'Systems Engineer']

    # Define the domains (people who can take each role)
    domains = {
        'Python Programmer 1': ['Peter', 'Jane', 'Bruce', 'Ciara'],
        'Python Programmer 2': ['Peter', 'Jane', 'Bruce', 'Ciara'],
        'AI Engineer 1': ['Jim', 'Juan', 'Peter', 'Anita'],
        'AI Engineer 2':['Jim', 'Juan', 'Peter', 'Anita'],
        'Web Designer': ['Juan', 'Mary', 'Anita'],
        'Database Admin': ['Jane'],
        'Systems Engineer': ['Jim', 'Mary', 'Bruce']}
    
    # Create the CSP
    csp = CSP(variables, domains)
    
    # Assign the main hard constraints
    domains['Python Programmer 1'] = ['Ciara'] 
    
    # Add the constraints
    csp.add_constraint(HiringLimitConstraint('Python Programmer 1','Python Programmer 2'))
    csp.add_constraint(HiringLimitConstraint('AI Engineer 1','AI Engineer 2'))
    csp.add_constraint(HiringLimitConstraint2(variables))

    # Find a solution
    solution = csp.backtracking_search()
    if solution is None:
        print("No solution found!")
    else:
        print("Solution :")
        people_hired = set(solution.values())
        print(people_hired)
        for keys,values in solution.items():
            print(keys, values)
        

Solution :
{'Juan', 'Jane', 'Ciara', 'Jim'}
Python Programmer 1 Ciara
Python Programmer 2 Jane
AI Engineer 1 Jim
AI Engineer 2 Juan
Web Designer Juan
Database Admin Jane
Systems Engineer Jim
